In [15]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from model import *
from model import make_model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

ImportError: cannot import name 'make_model'

In [32]:
d_model=50
DATA_PATH = '../data/test_data_'+str(d_model)+'d.pkl'
OUTPUT_PATH = '../output/PIT2015_02_attention.output'
MODEL_PATH = 'models/attention_epoch_3001.torch'

In [37]:
def run_model(data_path, output_path, model):
    with open(data_path, 'rb') as f:
        x0 = pickle.load(f)
        x1 = pickle.load(f)

    model = model.to(device)
    x0 = x0.to(device)
    x1 = x1.to(device)
    y_pred = model(x0, x1)
    y_pred = F.log_softmax(y_pred, dim=1)
    print(y_pred.size())
    
    with open(output_path, 'w') as f:
        for y in y_pred:
            if y[0] > y[1]:
                f.write('true\t')
            else:
                f.write('false\t')
            f.write(str(y[0].item()) + '\n')

In [38]:
def make_model(d_model, max_len, N=6, d_ff=256, h=5, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = AttentionEnc(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        c(position), d_model, max_len)
        
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [39]:
model = make_model(d_model, 18)
model.load_state_dict(torch.load(MODEL_PATH))

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:16: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


<All keys matched successfully>

In [40]:
run_model(DATA_PATH, OUTPUT_PATH, model)

torch.Size([838, 2])
